### Converting VCF to Newhybrids format..


###Step to filter Diagnostic loci from VCF first if available

In [1]:
wd = "/media/dan/34D5D1CE642D7E36/2013076_Hanfling_Bernd/Stacks/Stacks_analyses_TRIMMED/With_reference/Data_links/pstacks_all/NewHybrids_populations/Cru_gib/Cru_gib_pop_codes_noEP01_newnames.txt_dir/"

Diag_whitelist = open(wd+"/Diagnostic_snps.txt", 'r').readlines()[1:] ## this is in an annoying R format so needs some formatting
Diag_whitelist = [i.split()[1].split("\"")[1].split("_")[0] for i in Diag_whitelist]

#print Diag_whitelist[:10]

VCF = open(wd+"/Ho_filtered_altered.vcf", 'r').readlines()
Diagnostic_VCF = open(wd+"/Diagnostic_VCF.vcf", 'w')
for line in VCF:
    if line.startswith("#"):
        Diagnostic_VCF.write(line)
    elif line.split()[2] in Diag_whitelist:
        Diagnostic_VCF.write(line)
Diagnostic_VCF.close()


In [15]:
def VCF2NewHybs(infile):
    
    import os
    
    ## Convert Data to 1s and 2s ... and write to a separate file ###
    ## made file in bash from vcf by just cutting first couple of columns and getting rid of header lines ...
    
    wd = infile.rpartition("/")[0]
    temp = "%s/%s_NH.temp" % (wd, infile.rpartition("/")[2].rpartition(".")[0])
    temp_handle = open(temp, 'w')
    
    sample_file = "%s/%s_NH_individuals.txt" % (wd, infile.rpartition("/")[2].rpartition(".")[0])
    
    for line in open(infile, 'r').readlines():
        
        if line.startswith("#CHROM"):
            samples = line.split()[9:]
        
        if not line.startswith('#'):
            for sample in line.split():
                if sample.split(':')[0] == '1/1':
                    temp_handle.write('22\t')
                elif sample.split(':')[0] == '0/0':
                    temp_handle.write('11\t')
                elif sample.split(':')[0] == '0/1':
                    temp_handle.write('12\t')
                elif sample.split(':')[0] == '1/0':
                    temp_handle.write('21\t')
                elif sample.split(':')[0] == './.':
                    temp_handle.write('0\t')
            temp_handle.write('\n')
    
    temp_handle.close()
    
    ## write samples to file
    
    with open(sample_file, 'w') as sample_file_out:
        for i in samples:
            sample_file_out.write("%s\n" % i)

    ## Make final file
        
    outfile = "%s/%s_NEWHYBS.txt" % (wd, infile.rpartition("/")[2].rpartition(".")[0])
    out_handle = open(outfile, 'w')
    
    ### Get file header info ####

    with open(temp, 'r') as temp_handle:
        Number_of_samples = len(temp_handle.readlines()[0].split())
    
    with open(temp, 'r') as temp_handle:
        Number_of_loci = (len(temp_handle.readlines())-1)

    Loc_names = []
    for line in open(infile, 'r').readlines():
        if '#' not in line:
            Loc_names.append(line.split()[2])

    print 'Number of samples = '+str(Number_of_samples)
    print 'Number of loci = '+str(Number_of_loci+1)

    #### write file header ####

    out_handle.write("NumIndivs "+str(Number_of_samples)+"\nNumLoci "+str(Number_of_loci+1)+"\nDigits 1\nFormat Lumped\nLocusNames\t")
    out_handle.writelines("%s\t" % name for name in Loc_names)
    out_handle.write("\n")


    #### Transpose #####

    sample_number = 1
    with open(temp, 'rb') as temp_handle:
        lis = [x.split() for x in temp_handle]
        for x in zip(*lis[0:]): # (The "*" in this tells zip to take the elements in the "lis" argument, not the thing "lis" itself)
            out_handle.write("%s\t" % sample_number)
            sample_number += 1
            for y in x:
                out_handle.write(y+'\t')
                
            out_handle.write('\n')
    
    ## This transposing bit is very useful! Need to become more comfortable with this...

    out_handle.close()
    temp_handle.close()


    print "Done"
    
    
    
    

In [16]:
    VCF2NewHybs("/home/djeffrie/Dropbox/TPing_Bern/Eco_Evo_Research_module_TP/Carassius/Data/NewHybrids/Ccar_Caur/Ccar_Caur_ohnolog_filtered.vcf")

Number of samples = 203
Number of loci = 2728
Done


In [7]:
    VCF2NewHybs("/home/djeffrie/Dropbox/TPing_Bern/Eco_Evo_Research_module_TP/Carassius/Data/NewHybrids/Ccar_Cgib/Ccar_Cgib_ohnolog_filtered.vcf")

Number of samples = 206
Number of loci = 951
Done


In [17]:
    VCF2NewHybs("/home/djeffrie/Dropbox/TPing_Bern/Eco_Evo_Research_module_TP/Carassius/Data/NewHybrids/Ccar_Cypcarp/Ccar_Cyp_carp_ohnolog_filtered.vcf")

Number of samples = 186
Number of loci = 3853
Done


### ### Now transpose that file, and write it with headers to final file

In [3]:

    
nf_Td = open(wd+"/NewHybs_Ccar_Caur.txt", 'w')

### Get file header info ####

Number_of_samples = len(open(wd+"/NewHybs.temp", 'r').readlines()[0].split())
Number_of_loci = (len(open(wd+"/NewHybs.temp", 'r').readlines())-1)

Loc_names = []
for line in open(wd+"/Ccar_Caur_ohnolog_filtered.vcf", 'r').readlines():
    if '#' not in line:
        Loc_names.append(line.split()[2])

print 'Number of samples = '+str(Number_of_samples)
print 'Number of loci = '+str(Number_of_loci+1)

#### write file header ####

nf_Td.write("NumIndivs "+str(Number_of_samples)+"\nNumLoci "+str(Number_of_loci+1)+"\nDigits 1\nFormat Lumped\nLocusNames\t")
nf_Td.writelines("%s\t" % name for name in Loc_names)
nf_Td.write("\n")
            
            
#### Transpose #####

with open(wd+"/NewHybs.temp", 'rb') as the_file:
    lis = [x.split() for x in the_file]
    for x in zip(*lis[0:]): # (The "*" in this tells zip to take the elements in the "lis" argument, not the thing "lis" itself)
        for y in x:
            nf_Td.write(y+'\t')
        nf_Td.write('\n')
## This transposing bit is very useful! Need to become more comfortable with this...

nf_Td.close()


print "Done"


Number of samples = 203
Number of loci = 2728
Done
